## Preamble

Here I build a data processing pipeline using the <a href="https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce" target="_blank" style="text-decoration:underline;">Olist' (Brazilian E-Commerce) dataset</a>.
The dataset includes over 100,000 orders from 2016 to 2018, providing a comprehensive view of e-commerce transactions in Brazil.
It is a relational dataset, including 9 CSV tables linked by IDs.

The dataset includes several interconnected tables, which can be merged to create a rich dataset for analysis.



### Loading datasets

In [ ]:

import os
import pandas as pd
from pathlib import Path
import numpy as np

local_path = Path.cwd()

# path to data folder
data_folder = local_path / "data" / "Olist ecommerce dataset"

# Create a function to load all datasets located in "data_folder"
def load_all_csv_data(folder_path):
    dataframes = {}
    csv_files_list = list(data_folder.glob("*.csv")) #  Use Path.glob() to search for all ".csv" file in the folder
    for file_path in csv_files_list:
        file_name = file_path.name
        print(f"Loading {file_name}...")
        try:
            # Read the CSV file into a DataFrame
            # low_memory=False is used for better type inference on large files
            df = pd.read_csv(
                file_path,
                low_memory=False # low_memory=False is used for better type inference on large files
            )
            # Store the DataFrame using the file name (without the extension) as the key
            key_name = file_name.replace('.csv', '')
            dataframes[key_name] = df
        except Exception as e:
            print(f"Error loading {file_name}: {e}")
    return dataframes

datasets = load_all_csv_data(data_folder) # Create a 'data dictionary' that will hold all DataFrames found in the folder.
datasets.keys()

print("\n--- Loaded DataFrames ---")
# Print the keys and the shape of each loaded DataFrame to confirm
for name, df in datasets.items():
    print(f"'{name}': {df.shape[0]} rows x {df.shape[1]} columns")

### let's create individual datasets and give them handy names:
df_orders = datasets['olist_orders_dataset'].copy()
df_order_items = datasets['olist_order_items_dataset'].copy()
df_customers = datasets['olist_customers_dataset'].copy()
df_geolocation = datasets['olist_geolocation_dataset'].copy()
df_order_payments = datasets['olist_order_payments_dataset'].copy()
df_order_reviews = datasets['olist_order_reviews_dataset'].copy()
df_products = datasets['olist_products_dataset'].copy()
df_sellers = datasets['olist_sellers_dataset'].copy()
df_product_name_translation = datasets['product_category_name_translation'].copy()

### Relationships between tables

| Table                                           | Key                                   | Links to                                  | Description                        |
| ----------------------------------------------- | ------------------------------------- | ----------------------------------------- | ---------------------------------- |
| `df_orders`                                     | `order_id`                            | customers, order_items, reviews, payments | Core transaction table             |
| `df_order_items`                                | `order_id`, `product_id`, `seller_id` | products, sellers                         | Each row = one product in an order |
| `df_products`                                   | `product_id`                          | order_items                               | Product attributes                 |
| `df_customers`                                  | `customer_id`                         | orders                                    | Buyer demographics and location    |
| `df_sellers`                                    | `seller_id`                           | order_items                               | Seller info                        |
| `df_geolocation`                                | `zip_code_prefix`                     | customers/sellers                         | Lat-long per postal code           |
| `df_order_reviews`                              | `order_id`                            | —                                         | Ratings and comments               |
| `df_order_payments`                             | `order_id`                            | —                                         | Payment method(s)                  |

## Data Cleaning

#### Clean ORDERS

In [ ]:
# "df_orders" is the core transaction table
# it is linked to customers, order_items, reviews and payments via "order_id"

df_orders.dtypes # show all columns type
df_orders.order_status.unique() # check the different levels in 'order_status'
# column 4th to 8th are timestamps
timestamp_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
## convert dates column to datetime (pd.to_datetime)
for col in timestamp_cols: # Iterate through all timestamp columns
    df_orders[col] = pd.to_datetime(df_orders[col], errors='coerce') # Convert the data in each selected column to a proper datetime format; replace unparseable strings with NaT (Not a Time/missing value)
df_orders.dtypes # check if it works

## remove canceled orders:
df_orders = df_orders[df_orders['order_status'] != 'canceled']

##### Build delivery delay features

df_orders['delivery_delay'] = (df_orders['order_delivered_customer_date'] - df_orders['order_estimated_delivery_date']).dt.days
# Calculate the difference between the actual customer delivery date and the estimated delivery date,
# then convert the result to the total number of days (using ".dt.days").
df_orders.delivery_delay.head()
# A positive value means the delivery was late; a negative value means it was early.

# Calculate the total number of days elapsed between the moment the order was purchased and when it was delivered to the customer.
df_orders['purchase_to_delivery_days'] = (df_orders['order_delivered_customer_date'] - df_orders['order_purchase_timestamp']).dt.days
df_orders.purchase_to_delivery_days.head()

## check for missing values per columns
df_orders.isnull().sum()
## check how many order are NOT delivered (probably generating the NAs)
print(df_orders['order_status'].value_counts())

#### Clean ORDER_ITEMS

In [ ]:
# in "df_order_items", each row = one product in an order.
# it is linked to df_orders, df_products and df_sellers

print(df_order_items.dtypes) # check columns type
df_order_items['shipping_limit_date'] = pd.to_datetime(df_order_items['shipping_limit_date'], errors='coerce') # convert to proper datetime

# Calculate a new feature: the ratio of the shipping cost (freight_value) to the item's price.
df_order_items['shipping_cost_ratio'] = df_order_items['freight_value'] / df_order_items['price']
len(df_order_items[df_order_items.price == 0]) # check if some price are 0

# Aggregate at order_id level (sum of all items in same order)
df_order_items_agg = df_order_items.groupby('order_id').agg({
    'price': 'sum',           # Calculate the total price for all items within each order.
    'freight_value': 'sum',   # Calculate the total shipping cost for all items within each order.
    'order_item_id': 'count'  # Count the number of distinct items associated with each order (which equals the number of rows grouped).
}).rename(columns={'order_item_id': 'n_items'}).reset_index()
# ---> df_order_items_agg has one row per order_id. This is necessary for joining with the main df_orders table
# Rename the counted column to 'n_items' for clarity, and convert the 'order_item_id' from the index to a regular column (with "reset_index()").

#### Clean PAYMENTS

In [ ]:
# df_order_payments include info on payment value and method
# it is linked to df_orders via order_id

print(df_order_payments.dtypes) # check columns type

# Aggregate at order_id level (is one order has multiple payments -> aggregate)
    'payment_sequential': 'max',  # Get the maximum payment sequence number (indicating the total count of distinct payments made for the order).
df_order_payments_agg = df_order_payments.groupby('order_id').agg({
    'payment_value': 'sum',       # Calculate the total amount paid for each order.
    'payment_type': lambda x: x.mode().iloc[0] if len(x.mode()) else np.nan
    # Determine the MOST FREQUENT (mode) payment type for the order. If there's a tie or no payment, return the first mode found or NaN.
}).reset_index()
# Convert the 'order_id' back from the index (used for grouping) into a regular column.
# ---> df_order_payments_agg has one row per order_id.

#### Clean REVIEWS

In [ ]:
# df_order_reviews includes ratings and comments
# it is linked to 'df_orders' via 'order_id'

print(df_order_reviews.dtypes) # check columns type

# The 2 last column are time stamp, we convert them both to proper datetime
df_order_reviews['review_creation_date'] = pd.to_datetime(df_order_reviews['review_creation_date'], errors='coerce')
df_order_reviews['review_answer_timestamp'] = pd.to_datetime(df_order_reviews['review_answer_timestamp'], errors='coerce')

# Aggregate rating are the order_id level
df_order_reviews_agg = df_order_reviews.groupby('order_id')['review_score'].mean().rename('avg_review').reset_index()
# Group the reviews by unique order_id and calculate the average review_score for each order.
# Rename the resulting Series to 'avg_review' and convert 'order_id' back from the index to a column for merging.

#### Clean CUSTOMERS

In [ ]:
# df_customers includes buyer demographics and location
# It is linked to 'df_orders' via 'customer_id'

print(df_customers.dtypes) # check columns type

## is there any duplicate 'customer_unique_id'?
print(f"There is {int(df_customers.duplicated(subset='customer_id', keep=False).sum())} duplicates in 'customer_id'")

#### Clean SELLERS

In [ ]:
# df_sellers includes seller info
# It is linked to 'order_items' via 'seller_id'

print(df_sellers.dtypes) # check columns type

## is there any duplicate 'seller_id'?
print(f"There is {int(df_sellers.duplicated(subset='seller_id', keep=False).sum())} duplicates in 'df_sellers'")

#### Clean PRODUCTS

In [ ]:
# df_products includes product attributes
# It is linked to 'df_order_items' via 'product_id'

print(df_products.dtypes) # check columns type

# for now, we do nothing with this table

#### Clean GEOLOCATION

In [ ]:
# df_geolocation includes latitude and longitude per postal code
# It is linked to 'df_customers' and 'df_sellers' via zip code prefix

print(df_geolocation.dtypes) # check columns type

# for now, we do nothing with this table

#### Summary of clean tables

In [ ]:
print(f"orders: {df_orders.shape}")
print(f"order_items_agg: {df_order_items_agg.shape}")
print(f"payment_agg: {df_order_payments_agg.shape}")
print(f"review_agg: {df_order_reviews_agg.shape}")
print(f"customers: {df_customers.shape}")
print(f"products: {df_products.shape}")
print(f"sellers: {df_sellers.shape}")
print(f"geo: {df_geolocation.shape}")

### Merge Datasets

GOAL: Build an Analytical Base Table (ABT) by combining all cleaned datasets
We create two main tables:
1. orders_merged: ONE ROW PER ORDER (for order-level analysis)
2. df_order_items: ONE ROW PER PRODUCT (for item-level analysis)


#### Merge Order-Level Data

Create a comprehensive order table where each row represents ONE complete order
We start with df_orders as our base and progressively add information from other tables

In [ ]:
orders_merged = (
    df_orders  # BASE TABLE: Start with the core orders table (defines which rows we keep)

    # Add aggregated item information (total price, freight, number of items per order)
    .merge(df_order_items_agg, on='order_id', how='left')
    # Brings in: 'price' (sum), 'freight_value' (sum), 'n_items' (count

    # Add aggregated payment information (total payment value, payment type per order)
    .merge(df_order_payments_agg, on='order_id', how='left')
    # Brings in: 'payment_value' (sum), 'payment_sequential' (max), 'payment_type' (mode)

    # Add aggregated review information (average review score per order)
    .merge(df_order_reviews_agg, on='order_id', how='left')
    # Brings in: 'avg_review' (mean of all review scores for this order)

    # Add customer information (demographics and location)
    .merge(df_customers,
           left_on='customer_id',  # Column in 'df_orders'
           right_on='customer_id',  # match in 'df_customers'
           how='left')
    # Brings in: 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state'
)

# RESULT: orders_merged now has ONE ROW per order_id with complete order information
print(orders_merged.dtypes)

print(f"The 'orders_merged' table includes {orders_merged.shape[0]} individual orders, one per row.\n"
      f"Note that {df_customers.customer_unique_id.duplicated().sum()} orders were placed by customers who had ordered previously,\n"
      f" i.e. there are {df_customers.customer_unique_id.duplicated().sum()} duplicates in the column customer_unique_id.")



Note: One customer can place multiple orders, so:
- Number of rows = number of transactions/orders
- Number of unique customer_id = number of unique people who made purchases

The column 'customer_unique_id' in the raw customer table includes 3345 replicates
This is because 'customer_unique_id' identifies the person (the individual), and the duplicates represent the total number of repeat orders placed by existing customers.

#### Enrich Item-Level Data (Product + Seller Info)

In [ ]:
# Now we work on df_order_items to add product and seller details
# Each row here represents ONE PRODUCT within an order (not aggregated)

# STEP 1: Add product information (category name in Portuguese, weight)
order_items_merged = df_order_items.merge(
    df_products[['product_id', 'product_category_name', 'product_weight_g']],  # Select only needed columns
    on='product_id',  # Match by product_id
    how='left'  # Keep all items, even if product info is missing (fills with NaN)
)
# Brings in: 'product_category_name' (Portuguese), 'product_weight_g'

# STEP 2: Add seller information (location details)
order_items_merged = order_items_merged.merge(
    df_sellers,  # Contains: seller_id, seller_zip_code_prefix, seller_city, seller_state
    on='seller_id',  # Match by seller_id
    how='left'  # Keep all items, even if seller info is missing
)
# Brings in: 'seller_zip_code_prefix', 'seller_city', 'seller_state'

# STEP 3: Translate product category names from Portuguese to English
order_items_merged = order_items_merged.merge(
    df_product_name_translation,  # Lookup table: product_category_name (PT) → product_category_name_english (EN)
    left_on='product_category_name',  # Column in order_items_merged (Portuguese name)
    right_on='product_category_name',  # Column in translation table (Portuguese name)
    how='left'  # Keep all items, even if translation is missing
)
# NOTE: We use 'left_on' and 'right_on' here because both tables have the same column name
# After merge, we get: 'product_category_name_english' (the translated English name)

# STEP 4: Clean up duplicate column created during translation merge
order_items_merged.drop(columns=['product_category_name'], inplace=True)
# This removes the 'product_category_name' because we just want to keep 'product_category_name_english'

print(f"Item-level table shape: {order_items_merged.shape}")
# RESULT: order_items_merged now has ONE ROW per product with complete product + seller info

#### Add Geolocation Data (Seller & Customer Coordinates)

GOAL: Calculate the physical distance between seller and customer for each order
This requires getting latitude/longitude for both sellers and customers

In [ ]:
# STEP 1: Get seller coordinates by merging sellers with geolocation data
df_sellers_geo = df_sellers.merge(
    df_geolocation[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']],  # Select only the key + coordinate columns
    left_on='seller_zip_code_prefix',  # Seller's ZIP code (in df_sellers)
    right_on='geolocation_zip_code_prefix',  # Matching ZIP code (in df_geolocation)
    how='left'  # Keep all sellers, even if coordinates are missing
)

df_sellers_geo['seller_id'].duplicated().sum() # seller_id has duplicate!
# -->We keep only the first lat/lng per seller
df_sellers_geo_unique = df_sellers_geo.drop_duplicates(subset='seller_id')

# Rename columns for clarity (specify we're talking about seller coordinates)
df_sellers_geo_unique = df_sellers_geo_unique.rename(columns={
    'geolocation_lat': 'seller_lat',
    'geolocation_lng': 'seller_lng'
}, inplace=False)
# RESULT: df_sellers_geo has latitude/longitude for each seller

# STEP 2: Add seller coordinates to the item-level table
order_items_merged = order_items_merged.merge(
    df_sellers_geo_unique[['seller_id', 'seller_lat', 'seller_lng']],  # Select only the key (the ID) and coordinates
    on='seller_id',  # Match by seller_id
    how='left'  # Keep all items, even if seller coordinates are missing
)
# RESULT: Each product in order_items_merged now has its seller's coordinates

# STEP 3: Get customer coordinates by merging customers with geolocation data
df_customers_geo = df_customers.merge(
    df_geolocation[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']],  # Select the key and the coordinate columns
    left_on='customer_zip_code_prefix',  # Customer's ZIP code (in df_customers)
    right_on='geolocation_zip_code_prefix',  # Matching ZIP code (in df_geolocation)
    how='left'  # Keep all customers, even if coordinates are missing
).rename(columns={
    'geolocation_lat': 'customer_lat',
    'geolocation_lng': 'customer_lng'
})
# RESULT: df_customers_geo has latitude/longitude for each customer

# STEP 4: Add customer coordinates to orders
df_orders_geo = df_orders.merge(
    df_customers_geo[['customer_id', 'customer_lat', 'customer_lng']],  # Select only ID and coordinates
    on='customer_id',  # Match by customer_id
    how='left'  # Keep all orders, even if customer coordinates are missing
)
# RESULT: Each order in df_orders_geo now has its customer's coordinates

#### Calculate Distance Between Seller and Customer

CHALLENGE: One order can have multiple sellers (if buying from different sellers)
SOLUTION: For simplicity, we take the FIRST seller per order for distance calculation

In [ ]:
# STEP 1: Get one seller's coordinates per order (takes the first seller if multiple exist)
seller_coords_per_order = order_items_merged.groupby('order_id')[['seller_lat', 'seller_lng']].first().reset_index()
# This creates a table with ONE ROW per order_id containing one seller's coordinates

# STEP 2: Combine seller and customer coordinates in one table
orders_distance = df_orders_geo[['order_id', 'customer_lat', 'customer_lng']].merge(
    seller_coords_per_order,  # Contains: order_id, seller_lat, seller_lng
    on='order_id',  # Match by order_id
    how='left'  # Keep all orders, even if seller coordinates are missing
)
# RESULT: One row per order with both seller and customer coordinates

# STEP 3: Calculate geodesic distance (shortest distance on Earth's surface)
from geopy.distance import geodesic
from tqdm.auto import tqdm # Import tqdm for showing a progress bar
tqdm.pandas() # Activate tqdm's integration with Pandas

def calc_distance_vec(row):
    """
    Calculate distance in kilometers between seller and customer.
    Uses geodesic distance (accounts for Earth's curvature).
    Returns NaN if coordinates are missing.
    """
    try:
        # Create coordinate tuples: (latitude, longitude)
        seller_location = (row['seller_lat'], row['seller_lng'])
        customer_location = (row['customer_lat'], row['customer_lng'])
        # Calculate distance using geopy's geodesic function
        return geodesic(seller_location, customer_location).km
    except:
        # Return NaN if calculation fails (missing coordinates, invalid values, etc.)
        return np.nan

# Apply the distance calculation (and display a progress bar with 'progress_apply')
# orders_distance['distance_km'] = orders_distance.progress_apply(calc_distance_vec, axis=1) # !!! VERY LONG TO RUN !!!
# axis=1 means apply function to each ROW (not each column)

## save the distance calculation because it's long to run:
# dist = orders_distance['distance_km']
output_folder = local_path / "data"/ "Olist ecommerce dataset" / "processed_datasets"
# dist.to_csv(output_folder / "distance_km.csv", index=False)
orders_distance['distance_km'] = pd.read_csv(output_folder / "distance_km.csv", low_memory=False)

## Ensure that there is one row per order_id in orders_distance:
orders_distance_unique = orders_distance.groupby('order_id')[['customer_lat', 'customer_lng', 'seller_lat', 'seller_lng', 'distance_km']].first().reset_index()

# STEP 4: Merge the calculated distance back into the main order table
orders_merged = orders_merged.merge(
    orders_distance_unique[['order_id', 'distance_km']],  # Select only order_id and the distance
    on='order_id',  # Match by order_id
    how='left'  # Keep all orders, even if distance is missing
)

# FINAL RESULT: orders_merged now has distance_km for logistics analysis
print(f"Final orders_merged shape: {orders_merged.shape}")

#### Summary of merged tables

TWO MAIN TABLES CREATED:

1. orders_merged (ORDER-LEVEL):
   - One row per order
   - Contains: order details, aggregated items/payments/reviews, customer info, distance
   - Use for: "What's the average delivery time?", "Which states order most?", etc.

2. order_items_merged (ITEM-LEVEL):
   - One row per product within each order
   - Contains: item details, product info, seller info, seller coordinates
   - Use for: "Which categories sell best?", "Which sellers are most popular?", etc.

## Feature Engineering

GOAL: compute new features derived from the existing ones to enable richer analysis

### Order-Level Analysis
---> working on 'orders_merged' dataset

#### Time-Based Features

In [ ]:
# Extract temporal components for seasonality analysis
orders_merged['order_year'] = orders_merged['order_purchase_timestamp'].dt.year
orders_merged['order_month'] = orders_merged['order_purchase_timestamp'].dt.month
orders_merged['order_day_of_week'] = orders_merged['order_purchase_timestamp'].dt.dayofweek  # 0=Monday, 6=Sunday
orders_merged['order_hour'] = orders_merged['order_purchase_timestamp'].dt.hour
orders_merged['is_weekend'] = orders_merged['order_day_of_week'].isin([5, 6]).astype(int)  # 1 if Sat/Sun
orders_merged['is_holiday_season'] = orders_merged['order_month'].isin([11, 12]).astype(int)  # Nov-Dec

# Calculates the time between the purchase being made and the payment being officially approved.
# (= payment processing speed).
orders_merged['approval_delay_hours'] = (
    orders_merged['order_approved_at'] - orders_merged['order_purchase_timestamp']
).dt.total_seconds() / 3600

# Calculate the time elapsed between the order being approved and the package being picked up by the shipping carrier.
# (= seller's internal handling speed).
orders_merged['carrier_pickup_days'] = (
    orders_merged['order_delivered_carrier_date'] - orders_merged['order_approved_at']
).dt.days

#### Order Value & Economics

In [ ]:
# Average price per item
orders_merged['avg_price_per_item'] = orders_merged['price'] / orders_merged['n_items']

# Shipping cost as percentage of order value
orders_merged['freight_percentage'] = (orders_merged['freight_value'] / orders_merged['price']) * 100

# Total order value (price + freight)
orders_merged['total_order_value'] = orders_merged['price'] + orders_merged['freight_value']

# Revenue per km (efficiency metric)
orders_merged['revenue_per_km'] = orders_merged['price'] / orders_merged['distance_km']

# High-value order flag (e.g., top 10% of orders)
orders_merged['is_high_value'] = (
    orders_merged['price'] > orders_merged['price'].quantile(0.9)
).astype(int)

#### Delivery Performance

In [ ]:
# Create delivery performance categories: 'Early', 'On Time', or 'Late'
orders_merged['delivery_status'] = orders_merged['delivery_delay'].apply(
    lambda x: 'Early' if x < -2 # Assign 'Early' if the delivery delay is less than -2 days (meaning it arrived 3 or more days before the estimated date).
    else ('On Time' if x <= 2   # Assign 'On Time' if the delivery delay is between -2 days (inclusive) and +2 days (inclusive).
          else 'Late')          # Otherwise (if the delay is greater than 2 days), assign 'Late'.
)

# Late delivery flag
orders_merged['is_late'] = (orders_merged['delivery_delay'] > 0).astype(int)

# Delivery speed category (SUBJECTIVE CATEGORIZATION ?)
orders_merged['delivery_speed'] = pd.cut( # pd.cut() is used to segment continuous data into discrete intervals
    orders_merged['purchase_to_delivery_days'], # The numerical column representing the total days from purchase to customer delivery.
    bins=[0, 7, 14, 30, 999], # Define the boundaries (bins) for the categories, measured in days.
    labels=['Express', 'Fast', 'Standard', 'Slow'] # Assign a descriptive label to each resulting bin/interval.
)

#### Customer Satisfaction

In [ ]:
# Binary satisfaction (good vs bad reviews)
orders_merged['is_satisfied'] = (orders_merged['avg_review'] >= 4).astype(int)

# Review category
orders_merged['review_category'] = pd.cut(
    orders_merged['avg_review'],
    bins=[0, 2, 3, 4, 5],
    labels=['Poor', 'Fair', 'Good', 'Excellent']
)

#### Logistics Complexity

In [ ]:
## Distance categories for logistics planning
orders_merged['distance_category'] = pd.cut(
    orders_merged['distance_km'],
    bins=[0, 100, 500, 1000, 5000],
    labels=['Local', 'Regional', 'National', 'Long Distance']
)

## Complexity score (combining items + distance)
orders_merged['logistics_complexity'] = (
    orders_merged['n_items'] * orders_merged['distance_km'] / 100
)
# This calculates a composite score by multiplying the number of items by the delivery distance (in km) and scaling the result down by dividing by 100 (preventing the nb to become extremely large).
# This score represents the combined effort or cost required to fulfill the order.

### Item-Level Analysis
---> working on 'order_items_merged' dataset

#### Product Economics

In [ ]:
# Product contribution to order
# --> Calculate the proportion that the price of an individual item contributes to the total price of its respective order.
order_items_merged['item_share_of_order'] = order_items_merged.groupby('order_id')['price'].transform( # Group by 'order_id' and focus on 'price'
    lambda x: x / x.sum() # divides the price of each individual item (x) by the order's total price (x.sum())
) # Result: The 'transform' method broadcasts the result back to the original DataFrame, ensuring the new feature has the same length as the original 'order_items_merged' table.

# Weight-to-price ratio (shipping efficiency)
order_items_merged['weight_per_dollar'] = (
    order_items_merged['product_weight_g'] / order_items_merged['price']
)

# Freight efficiency per gram
order_items_merged['freight_per_gram'] = (
    order_items_merged['freight_value'] / order_items_merged['product_weight_g']
)

#### Product Categories

In [ ]:
# Heavy vs light products
order_items_merged['is_heavy_product'] = (
    order_items_merged['product_weight_g'] > order_items_merged['product_weight_g'].median()
).astype(int)

# Premium product flag (top 25% by price)
order_items_merged['is_premium'] = (
    order_items_merged['price'] > order_items_merged['price'].quantile(0.75)
).astype(int)

#### Seller Performance

In [ ]:
# Create seller-level aggregations, then merge back
seller_stats = order_items_merged.groupby('seller_id').agg({
    'order_id': 'nunique',  # Number of unique orders
    'price': 'mean'  # Average item price
}).rename(columns={'order_id': 'seller_order_count', 'price': 'seller_avg_price'}) # Rename the resulting aggregated columns to be descriptive.

order_items_merged = order_items_merged.merge(seller_stats, on='seller_id', how='left')
# Merge the calculated 'seller_stats' DataFrame back into the 'order_items_merged' DataFrame.
# Each item row now inherits the seller's total order count and average price.

# Top seller flag (e.g., sellers with 100+ orders)
order_items_merged['is_top_seller'] = (
    order_items_merged['seller_order_count'] >= 100
).astype(int)

#### Geographic Features

In [ ]:
# Same state transaction (seller and customer in same state)
# First need to merge customer_state into order_items_merged
order_items_merged = order_items_merged.merge(
    orders_merged[['order_id', 'customer_state']],
    on='order_id',
    how='left'
)

order_items_merged['is_local_transaction'] = (
    order_items_merged['seller_state'] == order_items_merged['customer_state']
).astype(int)

#### Customer Behavior

In [ ]:
# Calculate at the customer-level (merge back to orders_merged)
customer_features = orders_merged.groupby('customer_unique_id').agg({
    'order_id': 'count',  # Frequency
    'order_purchase_timestamp': 'max',  # Recency
    'price': 'sum'  # Monetary
}).rename(columns={
    'order_id': 'customer_order_count',
    'order_purchase_timestamp': 'customer_last_order',
    'price': 'customer_lifetime_value' # it's actually the Historical Customer Value (HCV) on the dataset period, but a good proxy of CLV
})

# Days since last order (as of dataset end date)
dataset_end_date = orders_merged['order_purchase_timestamp'].max()
customer_features['days_since_last_order'] = (
    dataset_end_date - customer_features['customer_last_order']
).dt.days

# Merge back in 'orders_merged'
orders_merged = orders_merged.merge(customer_features, on='customer_unique_id', how='left')
# Merges the three RFM features (count, last order date, and CLV) and the 'days_since_last_order' feature back into the main order table.
# *** ---> Note that, for instance, if a customer placed 5 orders,
#          there will be 5 rows in orders_merged containing the exact same customer_order_count and customer_lifetime_value.

# Customer segment
orders_merged['is_repeat_customer'] = (orders_merged['customer_order_count'] > 1).astype(int)
# Creates a binary flag (1 or 0) indicating whether the order was placed by a customer who has made more than one order in total.
# This feature is essential for analyzing loyalty and retention.

#### Product Popularity

In [ ]:
# Product-level aggregations
product_stats = order_items_merged.groupby('product_category_name_english').agg({
    'order_id': 'nunique', # Calculates the total number of unique orders placed for each product category (Frequency).
    'price': 'mean'  # Calculates the average price of all items sold within each product category (Monetary).
}).rename(columns={
    'order_id': 'category_order_count', # Renames the unique order count to a descriptive frequency metric.
    'price': 'category_avg_price' # Renames the mean price to the category's average value.
})

# Merge back in the item-level table
order_items_merged = order_items_merged.merge(
    product_stats,
    on='product_category_name_english', # Joins the summary table back to the item-level table using the category name as the key.
    how='left' # Ensures all item records remain in the table, with each item inheriting its category's statistics.
)

# Popular category flag (top 20% by order count)
order_items_merged['is_popular_category'] = (
    order_items_merged['category_order_count'] >
    order_items_merged['category_order_count'].quantile(0.8) # Calculates the value of the 80th percentile for the 'category_order_count' feature.
).astype(int)
# Creates a binary flag (1 or 0) indicating whether the category belongs to the top 20% of categories based on the total number of orders received.
# This discretizes the frequency metric into a clear 'popular' vs. 'not popular' feature for modeling.

#### Summary of engineered features

| Feature Type         | Business Use Case                                 |
|----------------------|---------------------------------------------------|
| Time-based           | Identify peak ordering times, seasonal trends     |
| Order economics      | Optimize pricing, understand shipping costs       |
| Delivery performance | Improve logistics, reduce late deliveries         |
| Customer satisfaction| Predict churn, improve service quality            |
| Seller performance   | Identify top sellers, optimize seller mix         |
| Geographic           | Regional marketing, warehouse placement           |
| Customer behavior    | Personalization, retention campaigns              |

These features will enable richer analysis like:
- Predicting delivery delays
- Identifying factors affecting customer satisfaction
- Segmenting customers/products for targeted strategies
- Optimizing logistics based on distance/weight/value

In [ ]:
print(orders_merged.shape)
print(orders_merged.isnull().sum())

print(order_items_merged.shape)
print(order_items_merged.isnull().sum())

### Create Customer-level Dataset

The 'orders_merged' table is at the order levels. We built a customer-level dataset derived from the 'orders_merged' table:

In [ ]:
### Aggregate to customer level

# Define end date for recency calculations
dataset_end_date = orders_merged['order_purchase_timestamp'].max()

df_customers_agg = orders_merged.groupby('customer_unique_id').agg(
    # --- Activity ---
    n_orders=('order_id', 'nunique'),
    first_order_date=('order_purchase_timestamp', 'min'),
    last_order_date=('order_purchase_timestamp', 'max'),

    # --- Monetary metrics ---
    total_revenue=('total_order_value', 'sum'),
    avg_order_value=('total_order_value', 'mean'),
    median_order_value=('total_order_value', 'median'),
    total_freight_paid=('freight_value', 'sum'),
    avg_freight_percentage=('freight_percentage', 'mean'),

    # --- Time / frequency metrics ---
    avg_days_between_orders=('days_since_last_order', 'mean'),

    # --- Satisfaction / experience ---
    avg_review=('avg_review', 'mean'),
    satisfaction_rate=('is_satisfied', 'mean'),

    # --- Delivery / logistics ---
    avg_delivery_delay=('delivery_delay', 'mean'),
    late_order_rate=('is_late', 'mean'),

    # --- Distance & geography ---
    avg_distance_km=('distance_km', 'mean'),
    most_frequent_state=('customer_state', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),
    most_frequent_city=('customer_city', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan),

    # --- Order value distribution ---
    pct_high_value_orders=('is_high_value', 'mean'),

    # --- Review category (optional) ---
    most_common_review_category=('review_category', lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
).reset_index()


### Add recency and lifetime metrics for each customer:

df_customers_agg['days_since_last_order'] = (dataset_end_date - df_customers_agg['last_order_date']).dt.days
# Recency: Calculates the number of days between the customer's most recent order and the final date in the dataset.
# A lower value indicates a more active customer.

df_customers_agg['customer_lifetime_days'] = (df_customers_agg['last_order_date'] - df_customers_agg['first_order_date']).dt.days
# Tenure/Longevity: Calculates the total duration (in days) between the customer's first and last order.
# # A higher value indicates a longer relationship with the platform.

# Loyalty / retention flags
df_customers_agg['is_repeat_customer'] = (df_customers_agg['n_orders'] > 1).astype(int)
# Creates a binary flag (1 or 0) indicating whether the customer has placed more than one order in total.
# This is a key feature for loyalty and retention studies.

# Average revenue per km (if distance exists)
df_customers_agg['revenue_per_km'] = df_customers_agg['total_revenue'] / df_customers_agg['avg_distance_km']
# Calculates how efficiently revenue is generated relative to the average shipping distance for that customer.
# This is useful for logistics and profitability analysis; customers with high revenue_per_km are logistically less costly to serve relative to their spend.

# RFM-like features
# df_customers_agg['recency'] = df_customers_agg['days_since_last_order']
# df_customers_agg['frequency'] = df_customers_agg['n_orders']
# df_customers_agg['monetary'] = df_customers_agg['total_revenue']


print(df_customers_agg.shape)

## Save processed datasets

In [ ]:
# Create an output folder if it doesn't exist
output_folder = local_path / "data"/ "Olist ecommerce dataset" / "processed_datasets"
output_folder.mkdir(parents=True, exist_ok=True) # Create the directory specified by 'output_folder'

### Save as CSV (human-readable, universally compatible)
# orders_merged.to_csv(output_folder / "orders_merged.csv", index=False)
# order_items_merged.to_csv(output_folder / "order_items_merged.csv", index=False)
# df_customers_agg.to_csv(output_folder / "customers_level_dataset.csv", index=False)
# print(f"✓ Datasets saved to: {output_folder}")
# print(f"  - orders_merged.csv: {orders_merged.shape}")
# print(f"  - order_items_merged.csv: {order_items_merged.shape}")
# print(f"  - customers_level_dataset.csv: {df_customers_agg.shape}")

### Save as Pickle (for Python-only workflows)
# orders_merged.to_pickle(output_folder / "orders_merged.pkl")
# order_items_merged.to_pickle(output_folder / "order_items_merged.pkl")